### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json



In [2]:
#set-up env file
load_dotenv()

True

In [3]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

In [4]:
#do a request to my API
#url = f"https://api.nytimes.com/svc/movies/v2/reviews/search.json?api-key+{nyt_api_key}"

### Access the New York Times API

In [5]:
# Set the base URL
url = f'https://api.nytimes.com/svc/search/v2/articlesearch.json?fq=section_name:"Movies" AND type_of_material:"Review" AND headline:"love"&api-key={nyt_api_key}&sort=newest&begin_date=20130101&end_date=20230531&field-name:(headline,web_url,snippet,source,keywords,pub_date,byline,word_count)'

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = url

# query_url = (
#   f"{url}api-key={nyt_api_key}&begin_date={begin_date}&end_date={end_date}"
#     + f'&fq={filter_query}&sort={sort}&fl={field_list}'
#     )



In [6]:

# Make a "GET" request and retrieve the JSON
reviews = requests.get(query_url).json()

# Add a twelve second interval between queries to stay within API query limits
#time.sleep(3)

# Try and save the reviews to the reviews_list
try:        
    # loop through the reviews["response"]["docs"] and append each review to the list
    print(reviews)
    
except:
    # Print the page number that had no results then break from the loop
    print("error")

{'status': 'OK', 'copyright': 'Copyright (c) 2024 The New York Times Company. All Rights Reserved.', 'response': {'docs': [{'abstract': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'web_url': 'https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html', 'snippet': 'A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.', 'lead_paragraph': 'A trashy treat coated in a high-art gloss, “The Attachment Diaries” gleefully kneads melodrama, noir, horror and sexual perversion into a pathological romance between two deeply damaged women.', 'print_section': 'C', 'print_page': '7', 'source': 'The New York Times', 'multimedia': [{'rank': 0, 'subtype': 'xlarge', 'caption': None, 'credit': None, 'type': 'image', 'url': 'images/2023/05/26/multimedia/attachment1-mbcw/attachment1-mbcw-articleLarge.jpg', 'height': 296, 'width': 600, 'legacy': {'xlarge': 'imag

In [7]:
# Create an empty list to store the reviews
reviews_list = [] 

# loop through pages 0-19
for page in range(0,20):
    # create query with a page number
    # API results show 10 articles at a time
    query_page_url = f"{query_url}&page={page}"
    
    # Make a "GET" request and retrieve the JSON
    reviews = requests.get(query_page_url).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(3)
    
    # Try and save the reviews to the reviews_list
    try:        
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in GET["response"]["docs"]:
            reviews_list.append(review)    
        # Print the page that was just retrieved
        print(f"Retrieved page {page}")
    except:
        # Print the page number that had no results then break from the loop
        print(f"No results for page {page}")
        break

No results for page 0


In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews_list[:5], indent=4))

In [ ]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_list_df=pd.json_normalize(reviews_list)
reviews_list_df.head()

In [ ]:
# Extract 'name' and 'value' from items in "keywords" column
reviews_list_df['keywords_names'] = reviews_list_df['keywords']
reviews_list_df['keywords_values'] = ''

# Extract 'name' and 'value'
def extract_keywords(keywords):
    if isinstance(keywords, str):
        extracted_keywords = []
        keyword_items = keywords.split(', ')
        for item in keyword_items:
        # Append the keyword item to the extracted_keywords list
            extracted_keywords.append(item)
        return ', '.join(extracted_keywords)
    else:
        return ''

# Fix the "keywords" column by converting cells from a list to a string
reviews_list_df["keywords"] = reviews_list_df["keywords"].apply(extract_keywords)


In [ ]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles = reviews_list_df["headline.main"].to_list()
titles

### Access The Movie Database API

In [ ]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [ ]:
# Create an empty list to store the results
tmdb_movies_list =[]

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 50

# Loop through the titles
for title in titles:
    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        try:
            # Get movie id
            movie_id = response ["results"][0]["id"]

            # Make a request for a the full movie details
            movie_url = f"https://api.themoviedb.org/3/movies/{movie_id}?api_key={tmdb_api_key}"
    
            # Execute "GET" request with url
            movie_response = requests.get(movie_url).json()
        
            # Extract the genre names into a list
            genres = []
            for genre in movie_response["genres"]:
                genres.append(genre["name"])
            # Extract the spoken_languages' English name into a list
            spoke_languages = []
            for language in movie_response["spoke_languages"]:
                spoke_languages.append(language["english_name"])
            # Extract the production_countries' name into a list
            production_countries = []
            for country in movie_response["production_countries"]:
                production_countries.append(country["name"])
            # Add the relevant data to a dictionary and
            # append it to the tmdb_movies_list list
            tmdb_movies_list.append({
                "title":movie_response["title"],
                "release_date": movie_response["release_date"],
                "runtime": movie_response["runtime"],
                "genres": ', '.join(genres),
                "spoken_languages": ', '.join(spoken_languages),
                "production_countries": ', '.join(production_countries),
                "vote_count": movie_response["vote_count"]
            })
            # Print out the title that was found
            print(f"Found movie: {title}")
        except:
            pass

In [ ]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))
            

In [ ]:
# Convert the results to a DataFrame
tmdb_movies_list_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_list_df.head()

### Merge and Clean the Data for Export

In [ ]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_list_df, tmdb_movies_list_df, on= "title") 
            

In [ ]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["keywords_names", "keywords_values"]

# Create a list of characters to remove
chars_to_remove = ['[', ']', "'"]

# Loop through the list of columns to fix
for column in columns_to_fix:

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for char in chars_to_remove:

        # Remove the character
        merged_df[column] = merged_df[column].str.replace(char, '')

# Display the fixed DataFrame
merged_df.head()

In [ ]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])
merged_df.head()
        

In [ ]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df.head()


In [ ]:
# Export data to CSV without the index
merged_df.to_csv("output/nyt_movie_reviews.csv", index=False)
print("Exported data to CSV!")
